In [ ]:
#  using Laplacian of Gaussians and scale-space extrema detection, we will detect and draw circles in the sunflower field image

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

im =cv.imread("images/the_berry_farms_sunflower_field.jpeg", cv.IMREAD_REDUCED_COLOR_4)
    